### **Importy**

Poniższy kod importuje różne biblioteki i moduły niezbędne do budowy i trenowania modelu klasyfikacji raka skóry:

- `os`: Interakcja z systemem operacyjnym.
- `pandas`: Analiza danych.
- `matplotlib.pyplot`: Wizualizacja danych.
- `tensorflow`: Budowanie i trenowanie modeli neuronowych.
- `numpy`: Obliczenia numeryczne.
- `ImageDataGenerator`: Augmentacja obrazów.
- `Sequential`: Model sekwencyjny Keras.
- `Warstwy Keras`: Konwolucyjne, pooling, spłaszczanie, gęste, dropout, global average pooling.
- `MobileNetV2`: Pretrenowany model do transfer learningu.
- `Adam`: Optymalizator.
- `train_test_split`: Podział danych na zestawy treningowe i testowe.
- `kagglehub`: Interakcja z platformą Kaggle.
- `EarlyStopping`: Zatrzymanie treningu przy braku poprawy.
- `load_model`: Ładowanie zapisanych modeli.
- `confusion_matrix, ConfusionMatrixDisplay`: Macierz pomyłek i jej wizualizacja.
- `PIL`: Manipulacja obrazami.
- `compute_class_weight`: Obliczanie wag klas dla niezbalansowanych danych.

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import kagglehub
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight

### **Pobieranie i przygotowanie danych**

Poniższy kod pobiera i przygotowuje dane do trenowania modelu klasyfikacji raka skóry:

- Pobieranie datasetu `HAM10000` z Kaggle.
- Szukanie i wczytywanie pliku metadanych `HAM10000_metadata.csv`.
- Szukanie katalogów z obrazami.
- Dodanie ścieżek do obrazów w dataframe.
- Podział danych na zbiory treningowy, walidacyjny i testowy.
- Obliczanie wag klas dla niezbalansowanych danych.
- Tworzenie generatorów danych z augmentacją dla zbioru treningowego oraz normalizacją dla zbiorów walidacyjnego i testowego.
- Wyświetlanie rozkładu klas w zbiorach danych.

In [ ]:
print("Pobieranie datasetu HAM10000...")
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
base_path = path[0]

print("\nSzukanie pliku metadata...")
metadata_file = None
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file == 'HAM10000_metadata.csv':
            metadata_file = os.path.join(root, file)
            print(f"Znaleziono plik metadata: {metadata_file}")
            break
    if metadata_file:
        break

print("\nWczytywanie metadanych...")
df = pd.read_csv(metadata_file)

print("\nSzukanie obrazów...")
images_dirs = []
for root, dirs, files in os.walk(base_path):
    for dir_name in dirs:
        if dir_name.startswith('HAM10000_images'):
            images_dirs.append(os.path.join(root, dir_name))

print(f"Znalezione katalogi z obrazami: {images_dirs}")

def find_image_path(image_id):
    for dir_path in images_dirs:
        img_path = os.path.join(dir_path, f'{image_id}.jpg')
        if os.path.exists(img_path):
            return img_path
    return None

df['filepath'] = df['image_id'].apply(find_image_path)

df = df.dropna(subset=['filepath'])

print("\nRozkład klas w zbiorze:")
print(df['dx'].value_counts())

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['dx'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['dx'], random_state=42)

print(f"\nRozmiary zbiorów:")
print(f"Rozmiar zbioru treningowego: {len(train_df)}")
print(f"Rozmiar zbioru walidacyjnego: {len(val_df)}")
print(f"Rozmiar zbioru testowego: {len(test_df)}")

img_size = (224, 224)
batch_size = 32

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['dx']),
    y=train_df['dx']
)
class_weight_dict = dict(zip(range(len(np.unique(train_df['dx']))), class_weights))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8,1.2],
    shear_range=0.2
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("\nRozkład klas w zbiorach:")
print("\nZbiór treningowy:")
print(train_df['dx'].value_counts())
print("\nZbiór walidacyjny:")
print(val_df['dx'].value_counts())
print("\nZbiór testowy:")
print(test_df['dx'].value_counts())

### **Wizualizacja danych**

Poniższy kod wizualizuje przykłady obrazów dla każdej klasy oraz wyświetla liczbę przypadków w każdej klasie:

- `dx_descriptions`: Słownik z opisami klas zmian skórnych.
- `examples`: Słownik z przykładowymi ścieżkami do obrazów dla każdej klasy.
- Tworzenie wykresu z przykładami obrazów dla każdej klasy.
- Wyświetlanie liczby przypadków w każdej klasie.

In [ ]:
dx_descriptions = {
    'akiec': 'Rogowacenie słoneczne / Rak kolczystokomórkowy (in situ)',
    'bcc': 'Rak podstawnokomórkowy',
    'bkl': 'Łagodne zmiany keratotyczne (keratosis benigna)',
    'df': 'Włókniak twardy (dermatofibroma)',
    'nv': 'Znamię melanocytowe',
    'mel': 'Czerniak',
    'vasc': 'Zmiany naczyniowe'
}

examples = {}
for dx in df['dx'].unique():
    example = df[df['dx'] == dx].iloc[0]
    examples[dx] = example['filepath']

plt.figure(figsize=(20, 10))

for idx, (dx, img_path) in enumerate(examples.items(), 1):
    try:
        img = Image.open(img_path)
        plt.subplot(2, 4, idx)
        plt.imshow(img)
        plt.title(f"{dx.upper()}\n{dx_descriptions[dx]}", fontsize=10)
        plt.axis('off')
    except Exception as e:
        print(f"Nie udało się załadować obrazu dla klasy {dx}: {e}")

plt.tight_layout()
plt.show()

print("\nLiczba przypadków w każdej klasie:")
class_counts = df['dx'].value_counts()
for dx in df['dx'].unique():
    print(f"\n{dx.upper()} - {dx_descriptions[dx]}")
    print(f"Liczba przypadków: {class_counts[dx]}")

### **Trenowanie modelu CNN**

Poniższy kod definiuje, kompiluje i trenuje model konwolucyjnej sieci neuronowej (CNN) od podstaw:

- Definicja modelu `Sequential` z warstwami konwolucyjnymi (`Conv2D`), pooling (`MaxPooling2D`), spłaszczającą (`Flatten`), gęstą (`Dense`) i dropout (`Dropout`).
- Kompilacja modelu z optymalizatorem `Adam`, funkcją straty `categorical_crossentropy` i metryką `accuracy`.
- Ustawienie callbacku `EarlyStopping` do zatrzymania treningu przy braku poprawy.
- Wyświetlenie podsumowania modelu.
- Trenowanie modelu na zbiorze treningowym z walidacją na zbiorze walidacyjnym, z użyciem wag klas i callbacku `EarlyStopping`.
- Zapisanie wytrenowanego modelu do pliku `model_cnn.h5`.

In [ ]:
print("Trenowanie modelu CNN od podstaw...")

model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model_cnn.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    mode='min',
    verbose=1
)

model_cnn.summary()

history_cnn = model_cnn.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

model_cnn.save('models/model_cnn.h5')

### **Wizualizacja wyników treningu**

Poniższy kod wizualizuje wyniki treningu modelu CNN:

- Tworzenie wykresu strat treningowych i walidacyjnych w zależności od epoki.
- Tworzenie wykresu dokładności treningowej i walidacyjnej w zależności od epoki.
- Wyświetlanie wykresów obok siebie dla lepszego porównania.

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_cnn.history['loss'], label='Strata treningowa')
plt.plot(history_cnn.history['val_loss'], label='Strata walidacyjna')
plt.title('Loss - CNN')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='Dokładność treningowa')
plt.plot(history_cnn.history['val_accuracy'], label='Dokładność walidacyjna')
plt.title('Accuracy - CNN')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()

plt.tight_layout()

plt.savefig('learning_curves/learning_curves_cnn.png')

plt.show()

### **Ewaluacja modelu CNN**

Poniższy kod ocenia model CNN na zbiorze testowym:

- `y_true`: Prawdziwe klasy z generatora testowego.
- `y_pred`: Przewidywane prawdopodobieństwa klas przez model.
- `y_pred_classes`: Przewidywane klasy (indeksy) przez model.
- Obliczanie i wyświetlanie macierzy pomyłek (`confusion_matrix`).
- Wyświetlanie macierzy pomyłek jako procentów.

In [ ]:
y_true = test_generator.classes
y_pred = model_cnn.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=test_generator.class_indices.keys())
cmd.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - CNN')
plt.show()

cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage = ConfusionMatrixDisplay(cm_percentage, display_labels=test_generator.class_indices.keys())
cmd_percentage.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - CNN')

plt.savefig('evaluation/percentage_confusion_matrix_cnn.png')

plt.show()

### **Trenowanie modelu z Transfer Learningiem (MobileNetV2)**

Poniższy kod definiuje, kompiluje i trenuje model z wykorzystaniem transfer learningu na bazie MobileNetV2:

- `base_model`: Pretrenowany model MobileNetV2 bez górnych warstw, z zamrożonymi wagami.
- Definicja modelu `Sequential` z bazowym modelem, globalną warstwą pooling, gęstą warstwą i dropout.
- Kompilacja modelu z optymalizatorem `Adam`, funkcją straty `categorical_crossentropy` i metryką `accuracy`.
- Wyświetlenie podsumowania modelu.
- Trenowanie modelu na zbiorze treningowym z walidacją na zbiorze walidacyjnym, z użyciem wag klas i callbacku `EarlyStopping`.
- Zapisanie wytrenowanego modelu do pliku `model_tl.h5`.

In [ ]:
print("Trenowanie modelu z Transfer Learningiem (MobileNetV2)...")

base_model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size[0], img_size[1], 3)
)

base_model.trainable = False

model_tl = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model_tl.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_tl.summary()

history_tl = model_tl.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

model_tl.save('models/model_tl.h5')

### **Wizualizacja wyników treningu (Transfer Learning)**

Poniższy kod wizualizuje wyniki treningu modelu z transfer learningiem:

- Tworzenie wykresu strat treningowych i walidacyjnych w zależności od epoki.
- Tworzenie wykresu dokładności treningowej i walidacyjnej w zależności od epoki.
- Wyświetlanie wykresów obok siebie dla lepszego porównania.

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_tl.history['loss'], label='Strata treningowa')
plt.plot(history_tl.history['val_loss'], label='Strata walidacyjna')
plt.title('Loss - Transfer Learning')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_tl.history['accuracy'], label='Dokładność treningowa')
plt.plot(history_tl.history['val_accuracy'], label='Dokładność walidacyjna')
plt.title('Accuracy - Transfer Learning')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()

plt.tight_layout()

plt.savefig('learning_curves/learning_curves_tl.png')

plt.show()

### **Ewaluacja modelu z Transfer Learningiem**

Poniższy kod ocenia model z transfer learningiem na zbiorze testowym:

- `y_true_tl`: Prawdziwe klasy z generatora testowego.
- `y_pred_tl`: Przewidywane prawdopodobieństwa klas przez model.
- `y_pred_classes_tl`: Przewidywane klasy (indeksy) przez model.
- Obliczanie i wyświetlanie macierzy pomyłek (`confusion_matrix`).
- Wyświetlanie macierzy pomyłek jako procentów.

In [ ]:
y_true_tl = test_generator.classes
y_pred_tl = model_tl.predict(test_generator)
y_pred_classes_tl = np.argmax(y_pred_tl, axis=1)

cm_tl = confusion_matrix(y_true_tl, y_pred_classes_tl)
cmd_tl = ConfusionMatrixDisplay(cm_tl, display_labels=test_generator.class_indices.keys())
cmd_tl.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Transfer Learning')
plt.show()

cm_percentage_tl = cm_tl.astype('float') / cm_tl.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage_tl = ConfusionMatrixDisplay(cm_percentage_tl, display_labels=test_generator.class_indices.keys())
cmd_percentage_tl.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - Transfer Learning')

plt.savefig('evaluation/percentage_confusion_matrix_tl.png')

plt.show()

### **Ewaluacja modeli CNN i Transfer Learning**

Poniższy kod ocenia modele CNN i Transfer Learning (TL) na zbiorze testowym:

- `y_true`: Prawdziwe klasy z generatora testowego.
- `y_pred_cnn`: Przewidywane prawdopodobieństwa klas przez model CNN.
- `y_pred_classes_cnn`: Przewidywane klasy (indeksy) przez model CNN.
- `y_pred_tl`: Przewidywane prawdopodobieństwa klas przez model TL.
- `y_pred_classes_tl`: Przewidywane klasy (indeksy) przez model TL.
- Obliczanie i wyświetlanie macierzy pomyłek (`confusion_matrix`) dla obu modeli.
- Wyświetlanie macierzy pomyłek jako procentów dla obu modeli.



In [ ]:
model_cnn = load_model('models/model_cnn.h5')
model_tl = load_model('models/model_tl.h5')

y_pred_cnn = np.argmax(model_cnn.predict(test_generator), axis=1)
y_pred_tl = np.argmax(model_tl.predict(test_generator), axis=1)

accuracy_cnn = accuracy_score(test_generator.classes, y_pred_cnn)
accuracy_tl = accuracy_score(test_generator.classes, y_pred_tl)

print(f'Dokładność modelu CNN: {accuracy_cnn:.2f}')
print(f'Dokładność modelu TL: {accuracy_tl:.2f}')

cm_cnn = confusion_matrix(test_generator.classes, y_pred_cnn)
cm_tl = confusion_matrix(test_generator.classes, y_pred_tl)

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

ConfusionMatrixDisplay(cm_cnn).plot(ax=ax[0], cmap='Blues')
ax[0].set_title('Macierz pomyłek - Model CNN')

ConfusionMatrixDisplay(cm_tl).plot(ax=ax[1], cmap='Blues')
ax[1].set_title('Macierz pomyłek - Model TL')

plt.show()

### **Funkcja predykcji**

Poniższa funkcja `predict_skin()` służy do klasyfikacji zdjęć zmian skórnych przy użyciu wytrenowanego modelu:

- Przyjmuje dwa parametry:
  - `model_path`: Ścieżka do zapisanego modelu (.h5)
  - `image_path`: Ścieżka do zdjęcia do analizy

- Wykonuje następujące operacje:
  - Wczytuje zapisany model
  - Przetwarza zdjęcie do formatu 224x224 pikseli
  - Normalizuje wartości pikseli do zakresu [0,1]
  - Dokonuje predykcji
  - Wyświetla wynik wraz ze zdjęciem

- Klasy wyjściowe:
  - akiec: Keratoza actinowa
  - bcc: Rak podstawnokomórkowy
  - bkl: Łagodne zmiany keratotyczne
  - df: Włókniak
  - mel: Czerniak
  - nv: Znamię melanocytowe
  - vasc: Zmiany naczyniowe

In [ ]:
def predict_skin(model_path, image_path):
    model = load_model(model_path)
    
    img = tensorflow.keras.preprocessing.image.load_img(
        image_path, 
        target_size=(224, 224)
    )
    img_array = tensorflow.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  
    img_array = numpy.expand_dims(img_array, axis=0)  
    

    prediction = model.predict(img_array)
    class_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
    predicted_class = class_names[numpy.argmax(prediction)]
    confidence = numpy.max(prediction) * 100
    

    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")
    

    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2f}%")
    plt.axis('off')
    plt.show()

# Example usage
# predict_skin_lesion('models/model_cnn.h5', 'path/to/your/image.jpg')